##### we run this on Google colab with *local runtime*

In [ ]:
!pip3 install matplotlib
!pip3 install scipy

In [ ]:
!pwd

/home/jonathan/cil_jonathan


In [ ]:
from os import walk
import cv2
import numpy as np

base = "/home/jonathan/CIL-street/data/original/all/"

groundtruth = []
for (dirpath, dirnames, filenames) in walk(base + 'groundtruth'):
    groundtruth.extend(filenames)
    break
groundtruth.sort()
print(groundtruth)

images = []
for (dirpath, dirnames, filenames) in walk(base + 'images'):
    images.extend(filenames)
    break
images.sort()
print(images)

['satImage_001.png', 'satImage_002.png', 'satImage_003.png', 'satImage_004.png', 'satImage_005.png', 'satImage_006.png', 'satImage_007.png', 'satImage_008.png', 'satImage_009.png', 'satImage_010.png', 'satImage_011.png', 'satImage_012.png', 'satImage_013.png', 'satImage_014.png', 'satImage_015.png', 'satImage_016.png', 'satImage_017.png', 'satImage_018.png', 'satImage_019.png', 'satImage_020.png', 'satImage_021.png', 'satImage_022.png', 'satImage_023.png', 'satImage_024.png', 'satImage_025.png', 'satImage_026.png', 'satImage_027.png', 'satImage_028.png', 'satImage_029.png', 'satImage_030.png', 'satImage_031.png', 'satImage_032.png', 'satImage_033.png', 'satImage_034.png', 'satImage_035.png', 'satImage_036.png', 'satImage_037.png', 'satImage_038.png', 'satImage_039.png', 'satImage_040.png', 'satImage_041.png', 'satImage_042.png', 'satImage_043.png', 'satImage_044.png', 'satImage_045.png', 'satImage_046.png', 'satImage_047.png', 'satImage_048.png', 'satImage_049.png', 'satImage_050.png',

In [ ]:
# https://jdhao.github.io/2019/02/23/crop_rotated_rectangle_opencv/ (how to crop in opencv)

def crop(img, angle, trans_x, trans_y):
    sidelen = 400
    cnt = np.array([
            [[sidelen/2, sidelen/2]],
            [[-sidelen/2, sidelen/2]],
            [[-sidelen/2, -sidelen/2]],
            [[sidelen/2, -sidelen/2]]
        ])
    rad = angle/180 * math.pi
    rot = np.array([[math.cos(rad), -math.sin(rad)],
          [math.sin(rad), math.cos(rad)]])
    print("rot")
    print(rot)
    A = np.dot(cnt, rot)
    A[:,:,0] += trans_x
    A[:,:,1] += trans_y

    if(np.min(A[:,:,0]) < 0):
      A[:,:,0] -= np.min(A[:,:,0])
    if(np.max(A[:,:,0]) > sidelen*2):
      A[:,:,0] -= (np.max(A[:,:,0])-sidelen*2)
    if(np.min(A[:,:,1]) < 0):
      A[:,:,1] -= np.min(A[:,:,1])
    if(np.max(A[:,:,1]) > sidelen*2):
      A[:,:,1] -= (np.max(A[:,:,1])-sidelen*2)

    rect = cv2.minAreaRect(A.astype("float32"))
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    clean = img.copy()
    cv2.drawContours(img, [box], 0, (0, 0, 255), 2)
    old_points = box.astype("float32")
    new_points = np.array([[sidelen, sidelen],
                        [0, sidelen],
                        [0, 0],
                        [sidelen, 0]], dtype="float32")

    M = cv2.getPerspectiveTransform(old_points, new_points)

    # directly warp the rotated rectangle to get the straightened rectangle
    warped = cv2.warpPerspective(clean, M, (sidelen, sidelen))

    return warped


In [ ]:
%matplotlib inline

In [ ]:
import random
import matplotlib.pyplot as plt
import math 
from scipy import ndimage

num_images = len(images)
print(num_images)

base = "/home/jonathan/CIL-street/data/original/all/"

four_images = []
four_gtis = []
for i in range(0, 600):
  img_a = []
  gti_a = []
  for a in range(0, 4):
    b = random.randint(0, num_images-1)
    angle = random.randint(0, 3) * 90
    gt = base + "groundtruth/" + groundtruth[b]
    gti = cv2.imread(gt)
    gti = ndimage.rotate(gti, angle)
    gti_a.append(gti)
    im = base + "images/" + images[b]
    img = cv2.imread(im)
    img = ndimage.rotate(img, angle)
    img_a.append(img)

  four_gti_top = np.concatenate((gti_a[0], gti_a[1]), axis=1)
  four_gti_bottom = np.concatenate((gti_a[2], gti_a[3]), axis=1)
  four_gti = np.concatenate((four_gti_top, four_gti_bottom), axis=0)
  imgplot = plt.imshow(four_gti)
  plt.show()

  four_img_top = np.concatenate((img_a[0], img_a[1]), axis=1)
  four_img_bottom = np.concatenate((img_a[2], img_a[3]), axis=1)
  four_img = np.concatenate((four_img_top, four_img_bottom), axis=0)
  imgplot = plt.imshow(four_img)
  plt.show()

  four_gtis.append(four_gti)
  four_images.append(four_img)

In [ ]:
sidelen = 400
num_generated = 1800
num_four_images = len(four_images)
base_save = "/home/jonathan/CIL-street/data/retiled/all/"

In [ ]:
import matplotlib.pyplot as plt
import random
import math

for i in range(0, num_generated):

    b = random.randint(0, num_four_images-1)
    four_img = four_images[b].copy()
    four_gti = four_gtis[b].copy()

    angle = random.random() * 360
    trans_x = sidelen/2 + random.random() * sidelen
    trans_y = sidelen/2 + random.random() * sidelen
    warped = crop(four_img, angle, trans_x, trans_y)
    cv2.imwrite(base_save + "images/img_" + str(i) + ".png", warped)

    warped_gt = crop(four_gti, angle, trans_x, trans_y)
    cv2.imwrite(base_save + "groundtruth/generated_" + str(i) + ".jpg", warped_gt)
    
    imgplot = plt.imshow(four_gti)
    plt.show()
    imgplot = plt.imshow(four_img)
    plt.show()
    imgplot = plt.imshow(warped_gt)
    plt.show()
    imgplot_warped = plt.imshow(warped)
    plt.show()